In [1]:
import os 
os.chdir('../')
%pwd

'd:\\pythonProjects\\SurgeSense'

# data transformation steps 

In [3]:
# entity
from dataclasses import dataclass
from pathlib import Path 
@dataclass
class DataTransformationConfig:
    root_dir: Path 
    data_path: Path 



In [4]:
# config 
from SurgeSense.constants import * 
from SurgeSense.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH):
        self.config=read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config=self.config.data_transformation 
        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        return data_transformation_config

In [7]:
# data transformation pipeline 
import pandas as pd 
C1=ConfigurationManager(CONFIG_FILE_PATH)
data_file=C1.get_data_transformation_config()
data=pd.read_csv(data_file.data_path)
data.head()

[2025-03-20 13:10:32,900: INFO :common : yaml file: config\config.yaml loaded successfully]
[2025-03-20 13:10:32,902: INFO :common : created directory at: artifacts]
[2025-03-20 13:10:32,903: INFO :common : created directory at: artifacts/data_transformation]


,distance,cab_type,destination,source,price,surge_multiplier,name,date_time,temp,location,clouds,pressure,rain,humidity,wind,day,hour,month
0,0.44,Lyft,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16 09:30:07.890000105,38.46,Haymarket Square,0.29,1022.25,0.0000,0.76,7.68,6,9,12
1,0.44,Lyft,North Station,Haymarket Square,11.0,1.0,Lux,2018-11-27 02:00:23.677000046,44.31,Haymarket Square,1.00,1003.17,0.1123,0.90,13.69,1,2,11
2,0.44,Lyft,North Station,Haymarket Square,11.0,1.0,Lux,2018-11-27 02:00:23.677000046,43.82,Haymarket Square,0.99,1002.59,0.0997,0.89,11.57,1,2,11
3,0.44,Lyft,North Station,Haymarket Square,26.0,1.0,Lux Black XL,2018-11-30 04:53:02.749000072,35.08,Haymarket Square,0.00,1013.71,0.0000,0.70,5.25,4,4,11
4,0.44,Lyft,North Station,Haymarket Square,9.0,1.0,Lyft XL,2018-11-29 03:49:20.223000050,37.58,Haymarket Square,0.42,998.64,0.0000,0.71,11.30,3,3,11


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer 

categorical_columns=['cab_type','destination','source','name','location']
numerical_columns=['distance','surge_multiplier']

In [ ]:
# components
import os 
from SurgeSense import logger
from sklearn.model_selection import train_test_split
import pandas as pd 

class DataTransformation: 
    def __init__(self,config:DataTransformationConfig):
        self.config=config


    def transform_data(self):
        data=pd.read_csv(self.config.data_path)
        